In [3]:
# return a dataframe
from train_test_val import main
# return train, train_labels, test, test_labels, val, val_labels
from train_test_val import train_test_val_split

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from classification_utilities import display_cm, display_adj_cm
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os

path = r"C:\Users\Dunbar\Dropbox\Work\FORCE"
infile = "train.csv"
infile_path = os.path.join(path, infile)

In [9]:
lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}

In [4]:
# df = main(infile_path, 51, 0.33)
df = pd.read_csv(infile_path, sep=';')

In [5]:
df["AI"] = df.DTC * df.RHOB

In [6]:
def numericalise(df, string):
    _string = df[string].unique()
    _dict = dict(zip(_string, range(len(_string))))
    df = df.replace(_dict)
    return df

df = numericalise(df, "FORMATION")

df = numericalise(df, "GROUP")

In [7]:
curves = ['DTC', 'FORMATION', 'GR', 'GROUP', 'NPHI',
          'PEF', 'RDEP', 'RHOB', 'RMED', 'RSHA', 'AI', 'FORCE_2020_LITHOFACIES_LITHOLOGY']
# curves = ['RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'AI']
# curves = ['RSHA', 'RMED', 'RDEP', 'GR', 'NPHI', 'PEF', 'AI']
# curves = ['RDEP', 'GR', 'NPHI', 'PEF', 'AI']

In [10]:
from sklearn.utils import shuffle
X = df[curves]
X = shuffle(X)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

y = X.pop('FORCE_2020_LITHOFACIES_LITHOLOGY')
y = y.map(lithology_numbers)
X = scaler.fit_transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [12]:
A = np.load('../penalty_matrix.npy')

def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]


def make_model(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train)
    print("Accuracy on training set: {:.3f}".format(model.score(X_train, y_train)))
    print("Accuracy on test set: {:.3f}".format(model.score(X_test, y_test)))
    print("Feature importances:\n{}".format(model.feature_importances_))
    predictions = model.predict(X_test)
    conf = confusion_matrix(y_test, predictions)
    print(display_cm(conf, facies_labels, hide_zeros=True))
    #print(confusion_matrix(y_test, predictions))
    print(score(y_test.values, predictions))

facies_labels = ['Sst', 'S-Sh', 'Sh', 'Mrl', 'Dol', 'Lst', 'Chk', 'Hal', 'Anh', 'Tuf', 'Coa', 'Bmt']

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
model = RandomForestClassifier(random_state=0)
make_model(model, X_train, y_train, X_test, y_test)

C:\Users\Dunbar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy on training set: 0.996
Accuracy on test set: 0.927
Feature importances:
[0.082893   0.09631545 0.20361503 0.08113734 0.08925217 0.06146562
 0.08465008 0.07475005 0.08239673 0.06172864 0.08179588]
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst 15466   639   708    12     1    47     1                 6    16       16896
     S-Sh   929 12256  1642    14          35                       2     3       14881
       Sh   512  1222 69931   125     4   120           6          60    22       72002
      Mrl    31    34   276  2927         138    10                                3416
      Dol    15    12    65     3    63     5                 2                     165
      Lst   238   189   806   156     3  4321    40                19              5772
      Chk           1     1     2          56  1005                                1065
      Hal                 5                           804     1                  

In [15]:
model = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',#'gini',
    max_depth=30, #None
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 1.000
Accuracy on test set: 0.936
Feature importances:
[0.08481275 0.09920064 0.1956954  0.11348929 0.09586579 0.05646047
 0.0752347  0.07447915 0.07597601 0.05534936 0.07343643]
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst 15513   593   725     5          31     1                 6    22       16896
     S-Sh   718 12433  1684     5     1    31                       5     4       14881
       Sh   409   853 70422   114     2   110           4     1    65    22       72002
      Mrl    16    33   215  3015         130     7                                3416
      Dol    12    12    70     3    58     5                 4     1               165
      Lst   185   160   767   130        4463    42                25              5772
      Chk           1     1     1          43  1019                                1065
      Hal                 5                           805                        

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [16]:
model = RandomForestClassifier(
    n_estimators=1000,
    criterion='entropy',#'gini',
    max_depth=20, #None
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.975
Accuracy on test set: 0.928
Feature importances:
[0.0836953  0.10603964 0.20216822 0.12408311 0.09497296 0.05381873
 0.06988481 0.07135817 0.07052741 0.05369282 0.06975883]
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst 15310   666   855     6          29     1                 6    23       16896
     S-Sh   799 11845  2194     9     1    23                       5     5       14881
       Sh   390   895 70409   120          91           6     1    69    21       72002
      Mrl    19    32   256  2961         141     7                                3416
      Dol    12    11    78     3    51     5                 4     1               165
      Lst   193   172   843   128        4362    49                25              5772
      Chk                 2     1          47  1015                                1065
      Hal                 5                           805                        